## Clickstream Processing

Load clickstream JSONs from ADLS & clean fields to match desired output.

* Reads JSONs from ADLS
* Cleanses fields to match desired schema
* Saves to ADLS as Parquet


In [53]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

In [4]:
# load data

df = spark.read.json("abfss://{FILE_SYSTEM_NAME}@{DATA_LAKE_NAME}.dfs.core.windows.net/synapse/workspaces/clickstream/new/")

In [ ]:
# clean up fields

delimiter ='-----'

df_final = df \
    .withColumn('Body', regexp_replace('Body','[^0-9A-Za-z_.]','-')) \
    .withColumn('event_time', from_unixtime(unix_timestamp('EnqueuedTimeUtc', 'MM/dd/yyy hh:mm:ss aa'),'yyyy-MM-dd HH:mm:ss.SSS')) \
    .withColumn('event_type', expr('case when Body like "%ShoppingCart%" then "purchase" else "view" end')) \
    .withColumn('product_id', lower(split('Body',delimiter).getItem(5))) \
    .withColumn('category_id',  when(lower(split('Body',delimiter).getItem(11))!="",lower(split('Body',delimiter).getItem(11)))) \
    .withColumn('category_code', when(lower(split('Body',delimiter).getItem(13))!="",lower(split('Body',delimiter).getItem(13)))) \
    .withColumn('brand', when(lower(split('Body',delimiter).getItem(7))!="",lower(split('Body',delimiter).getItem(7)))) \
    .withColumn('price', split('Body',delimiter).getItem(9)) \
    .withColumn('user_session', when(split('Body',delimiter).getItem(15)!="",split('Body',delimiter).getItem(15))) \
    .withColumn('user_id', split('Body',delimiter).getItem(1).cast(IntegerType())) \
    .drop('Body','EnqueuedTimeUtc','Offset','SequenceNumber') \
    .filter(col('product_id').isNotNull())

In [61]:
# save to adls

df_final.write.mode('append').format('delta').option('header','true').save('abfss://{FILE_SYSTEM_NAME}@{DATA_LAKE_NAME}.dfs.core.windows.net/synapse/workspaces/clickstream/processed/')